In [1]:
from Swarm import Swarm
import Helper

import sys
import copy
import time
import argparse
import os

import numpy as np
from scipy import stats

from multiprocessing import Pool
from multiprocessing import cpu_count
from itertools import repeat
from tqdm import tqdm

from Helper import Write_Log

In [2]:
def lossFunction(tar,b,lossFunc = 0):
    if (lossFunc == 1):
        newCost = np.sum( (b-tar)**2, axis=1 )/len(b)#MSE
    elif(lossFunc == 2):
        newCost = np.sqrt(np.sum( (b-tar)**2, axis=1 ))#RMSE
    elif(lossFunc == 3):
        #Heuber
        delta = 0.1
        y = tar
        yHat = b
        newCost = np.sum(np.where(np.abs(y-yHat) < delta,.5*(y-yHat)**2 , delta*(np.abs(y-yHat)-0.5*delta)))
    else :
        newCost = np.sum( (b-tar)**2)#SSE
    return newCost

In [3]:
# Prints statistics of the current swarm
def Print_Stats(swarm, contact, pointCount, i, outFilePtr, convFact):
    pers = stats.pearsonr(swarm.gBest[2], contact[:,3])
    spear = stats.spearmanr(swarm.gBest[2], contact[:,3])
    spearIF = stats.spearmanr(swarm.gBest[2], contact[:,2])

    error = np.sqrt( (1/pointCount) * np.sum( (swarm.gBest[2]-contact[:,3])**2 ) )

    #print('id: ' + str(swarm.id) + 
    #    ' itt: ' + str(i) + 
    #    ' Cost: ' + str(swarm.gBest[1]) + 
    #    ' Pearson: ' + str(pers[0]) + 
    #    ' Spearmen: ' + str(spear[0]) +
    #    ' IFSpear: ' + str(spearIF[0]) +
    #    ' error: ' + str(error))
    thisOutFilePtr = 'outputFolder/'+outFilePtr +str(convFact)
    

def Write_Stats(swarm, contact, outFilePtr):
    Helper.Write_Output(outFilePtr, swarm.gBest[0])

# Performs one operation and prints statistics of current swarm
def One_Move(ittCount, swarm, contact, pointCount, threshold,  outFilePtr, convFact):
    saveGBestCost = float('inf')
    totTime = 0


    for i in range(ittCount):
        if (i%1000 == 0) and (swarm.gBest is not None):
            #error = np.sqrt( (1/pointCount) * np.sum( (swarm.gBest[2]-contact[:,3])**2 ) )
            error = lossFunction(contact[:,3],swarm.gBest[2])#np.sum( (swarm.gBest[2]-contact[:,3])**2 )
            Print_Stats(swarm, contact, pointCount, i, outFilePtr, convFact)
            
                

            if (np.abs(saveGBestCost - error)) >= threshold:
                saveGBestCost = error
            else:
                return i, totTime

        operation(i, swarm)


    return i

# Performs a single PSO pass: Velocity calculation, update position, get new cost
def operation(i, swarm):
    swarm.Calc_Vel(ittCount,i)
    swarm.Update_Pos(i)
    swarm.Cost()

# Optimizes single swarm
def Optimize(inFilePtr, outFilePtr, convFact,constraint,points,zeroInd,lossFunc = 0):
    dist = 1.0 / (constraint[:,2]**convFact)
    constraint = np.insert(constraint,3, dist ,axis=1)
    
    swarm = Swarm(constraint, len(points), randVal=randRange, swarmSize=swarmSize, zeroInd=zeroInd,lossFunc = lossFunc )

    ittFin = One_Move(ittCount, swarm, constraint, len(points), threshold,  outFilePtr, convFact)
    
    #pbar.update(1)
    return (stats.pearsonr(swarm.gBest[2], constraint[:,3])[0], 
                    stats.spearmanr(swarm.gBest[2], constraint[:,3])[0], 
                    lossFunction(constraint[:,3],swarm.gBest[2]),
                    ittFin,
                     swarm.id, swarm)

# Runs in paralel if passed multiple rangeSpace
def Par_Choice(inFilePtr, outFilePtr, alpha,lossFunc = 0):
    contact, points, zeroInd = Helper.Read_Data(inFilePtr, alpha)
    
    bestSwarm = None
    if 1==1:
        convStore = []
        alphas = np.array(range(int(alpha[0]),int(alpha[1]),int(alpha[2])))/100
        pool = Pool(processes=PROC_COUNT)
        #pbar = tqdm(total=len(alphas))#progress bar
        swarms = pool.starmap(Optimize,  zip(repeat(inFilePtr), repeat(outFilePtr), 
                                             alphas, 
                                             repeat(contact), repeat(points), repeat(zeroInd), repeat(lossFunc)))

        pool.close()
        pool.join()

        #swarms = sorted(swarms, key=lambda x: x[1])
        
        iforapl = 0
        for swarm in swarms:
            print(str(swarm[-1]) + ' ' + str(swarm[1]))
            contact = np.insert(contact,3, 1.0 / (contact[:,2]**alphas[iforapl]) ,axis=1)
            thisOutFile = 'outputFolder/'+outFilePtr+"_alpha_"+str(alphas[iforapl])
            
            Write_Stats(swarm[len(swarm)-1], contact, thisOutFile)
            convStore.append(swarm)
            if (bestSwarm is None) or (swarm[1] > bestSwarm[1]):
                bestSwarm = swarm
                swarmForPDB = swarm[len(swarm)-1]
                bestAlpha = alphas[iforapl]
            iforapl += 1
    else:#single thread
        bestSwarm = Optimize( inFilePtr, outFilePtr, alpha)
    contact = np.insert(contact,3, 1.0 / (contact[:,2]**bestAlpha) ,axis=1)
    outFilePtr = 'outputFolder/'+outFilePtr+"_best"
    print(bestSwarm)
    Write_Stats(swarmForPDB, contact, outFilePtr)

    return bestSwarm

def Full_List( inputFilePtr, outFilePtr , alpha, lossFunc = 0):
    convStore = []
    
    convStore.append(Par_Choice( inputFilePtr, outFilePtr, alpha, lossFunc))
    print("pearson:" + str(convStore[0][0]) + " spearman:"+
          str(convStore[0][1]) + " rmse:" + str(convStore[0][2]))

    #Helper.Write_List(convStore, outFilePtr)
    return convStore

In [4]:
sys.setrecursionlimit(10000)
PROC_COUNT = cpu_count()

rangeSpace = [] # Max scaling factor. Needs to be optimized for each specific dataset. Use two values [one, two] to multithread through a range of those two values at a interval of 5000


# Arguments for running program
# python3 ParticleChromo3D.py <input_data> <other_parameter>

randRange = 1.0
swarmSize = 15
ittCount = 30000
threshold = 0.000001

if len(rangeSpace) == 0:
    rangeSpace.append(20000)

if len(rangeSpace) > 2 and (rangeSpace[0] == rangeSpace[1]):
    rangeSpace.pop()
    
if not os.path.exists('outputFolder'):
    os.makedirs('outputFolder')



theseAlphas = np.array([0.1, 2.0, 0.2])*100
theAlphas = np.array(range(int(theseAlphas[0]),int(theseAlphas[1]),int(theseAlphas[2])))/100



In [5]:
lossFunctionChoice = 3#1 == MSE 2 = rmse 3 == Huber

allSpears = [] 
for i in range(24):
    chromosomeRunning = i+1
    inFilePtr = '../input-and-models/Input/GM12878_input/KR_1mb/chr'+str(chromosomeRunning)+'_matrix.txt'
    outFilePtr = './lossFunc/chr'+str(chromosomeRunning)

    print(inFilePtr)

    fout = inFilePtr + ".stripped"
    clean_lines = []
    f= open(inFilePtr, "r")
    lines = f.readlines()
    for l in lines:
        res = str(" ".join(l.split()))
        clean_lines.append(res)
    f.close()

    with open(fout, "w") as f:
        f.writelines('\n'.join(clean_lines))
    f.close()


    start = time.time()
    print("Seconds since epoch =", start)
    outputOfSwarm = Full_List( inFilePtr+".stripped", outFilePtr, theseAlphas, lossFunctionChoice)[0]
    print(outputOfSwarm)

    bestSpearm = outputOfSwarm[1]
    bestCost = outputOfSwarm[2]
    bestAlpha = theAlphas[outputOfSwarm[4]]
    bestPearsonRHO = outputOfSwarm[0]


    print("Input file: ", inFilePtr)
    print("Convert factor:: ",bestAlpha)
    print("SSE at best spearman : ", bestCost)    
    print("Best Spearman correlation Dist vs. Reconstructed Dist  : ", bestSpearm) 
    print("Best Pearson correlation Dist vs. Reconstructed Dist: ", bestPearsonRHO) 
    Write_Log("outputFolder/bestAlpha.log", inFilePtr, bestAlpha, bestCost, bestSpearm, bestPearsonRHO)
    net = time.time() - start
    allSpears.append(bestSpearm)
    print("time : ", net)
    
    print("All spears : ", allSpears)
    

../input-and-models/Input/GM12878_input/KR_1mb/chr1_matrix.txt
Seconds since epoch = 1618553511.2303438
<Swarm.Swarm object at 0x7fa26a0c2a00> 0.8774783710034095
<Swarm.Swarm object at 0x7fa26a0c2d30> 0.9427150961013707
<Swarm.Swarm object at 0x7fa26a0c2910> 0.9462844819593886
<Swarm.Swarm object at 0x7fa26a0c2be0> 0.908052348447351
<Swarm.Swarm object at 0x7fa26a0c2820> 0.8533146737750106
<Swarm.Swarm object at 0x7fa26a0c2a60> 0.6109247513144321
<Swarm.Swarm object at 0x7fa26a0c2c40> 0.16092357083638667
<Swarm.Swarm object at 0x7fa26a0c2d90> 0.06863397667596531
<Swarm.Swarm object at 0x7fa26a0c2700> -0.006189618193415256
<Swarm.Swarm object at 0x7fa26a0c2790> 0.01163359228128102
(0.9285415066436684, 0.9462844819593886, 1.7568602779213351, 29999, 1, <Swarm.Swarm object at 0x7fa26a0c2910>)
pearson:0.9285415066436684 spearman:0.9462844819593886 rmse:1.7568602779213351
(0.9285415066436684, 0.9462844819593886, 1.7568602779213351, 29999, 1, <Swarm.Swarm object at 0x7fa26a0c2910>)
Input file

FileNotFoundError: [Errno 2] No such file or directory: '../input-and-models/Input/GM12878_input/KR_1mb/chr24_matrix.txt'

In [ ]:
for i in allSpears:
    print(i)